# 🚕 NYC Taxi Data Analysis - January 2024
> **Project:** Big Data Engineering Mini-Challenge  
> **Goal:** ค้นหาประเภท Taxi ที่มียอดการใช้งาน (Rides) สูงที่สุดในเดือนมกราคม 2024

---

## 🏗️ Detailed Data Pipeline (AWS Implementation)
การทำงานของโค้ดใน Notebook นี้แบ่งเป็น 5 ขั้นตอนตามหลักการ **Modern Data Lakehouse**:

1.  🚀 **Stage 1: AWS CONFIGURATION & DEPENDENCIES**
    * **AWS_ACCESS_KEY**
    * **AWS_SECRET_KEY**
    * **AWS_SESSION_TOKEN**
    * **REGION_NAME**
    * **BUCKET_NAME**
2.  📥 **Stage 2: INGEST (to AWS S3)**
    * **AWS Service:** Amazon S3 (Simple Storage Service)
    * **Action:** โค้ดจะดาวน์โหลดไฟล์ Parquet จาก Source และ Upload ขึ้น S3 Bucket ทันที
    * **Technique:** จัดเก็บแบบ **Partitioned Folder** (`/type=yellow`, `/type=green`) เพื่อแยกประเภทข้อมูลตั้งแต่ระดับ Storage

3.  🏗️ **Stage 3: DEFINE SCHEMA (via Glue/Athena)**
    * **AWS Service:** AWS Glue Data Catalog
    * **Action:** ใช้คำสั่ง SQL (`CREATE EXTERNAL TABLE`) เพื่อสร้าง "ตารางเสมือน" ชี้ไปยังไฟล์ใน S3
    * **Concept:** **"Schema-on-Read"** — เราไม่ได้แปลงไฟล์ข้อมูลจริงๆ แต่เราสร้าง Metadata เพื่อบอกให้ Athena รู้วิธีอ่านข้อมูลนั้น

4.  ⚡ **Stage 4: COMPUTE & AGGREGATE (Serverless)**
    * **AWS Service:** AWS Athena
    * **Action:** ส่งคำสั่ง SQL (`SELECT count(*) ... UNION ALL ...`) ไปประมวลผลบน Cloud
    * **Benefit:** ไม่ต้องโหลดข้อมูล 23 ล้านแถวลง RAM เครื่องตัวเอง (No Out-of-Memory Errors) และประมวลผลเร็วด้วย Presto Engine

5.  🔄 **Stage 5: RETRIEVE RESULTS**
    * **AWS Service:** AWS SDK (Boto3)
    * **Action:** Python รอรับเฉพาะ "ผลลัพธ์สุดท้าย" (Aggregated Data) ที่เล็กมากๆ กลับมา
    
6.  📑 **Stage 6: REPORTING**
    * **Tool:** Pandas & Jupyter Notebook
    * **Action:** นำผลลัพธ์มาจัดเรียง (Ranking) และแสดงผลเป็นตารางสรุป

---

## ☁️ Cloud Architecture: Modern Data Lakehouse

เราเปลี่ยนจากการประมวลผลแบบ Local Processing มาเป็น **AWS Serverless Stack** เพื่อรองรับ **Scalability** และ **Cost Efficiency** โดยใช้แนวคิดหลักดังนี้:

### 💡 Key Concepts: ELT > ETL
* **ELT (Extract, Load, Transform):** เน้นการดึงข้อมูลและ "Load" ลง Storage (S3) ทันทีเพราะราคาถูกมาก แล้วค่อย "Transform" ภายหลังเมื่อต้องการใช้งาน
* **Serverless Querying:** ใช้ Athena ยิง Query เพื่อประมวลผลข้อมูลโดยตรงจากไฟล์ Parquet โดยไม่ต้อง Provision Server ทิ้งไว้ (No-Ops)

### 🛠️ Tech Stack Selection
| Service | Role | Key Benefit |
| :--- | :--- | :--- |
| **AWS S3** | **Storage** | เก็บไฟล์ Parquet (Decoupled Storage) ทนทานสูงและประหยัดกว่า Local Disk |
| **AWS Glue** | **Metadata** | ทำหน้าที่เป็น Data Catalog เก็บ Schema กลาง ไม่ต้อง Hardcode |
| **AWS Athena** | **Compute** | SQL Query Engine แบบ Serverless คิดเงินตามข้อมูลที่ Scan เท่านั้น |

---

## 📐 Architecture Diagram

แผนภาพการไหลของข้อมูล (Data Flow) ตั้งแต่ต้นทางจนถึงการออกรายงาน:

In [ ]:
print("""
══════════════════════════════════════════════════════════════════════════════════════════════════
                           🗺️  DATA PIPELINE MAP (NYC Taxi Data - AWS Edition)
══════════════════════════════════════════════════════════════════════════════════════════════════

┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  [STAGE 1: SOURCE]                                                                              │
│  ─────────────────                                                                              │
│        🌐 INTERNET SOURCE (NYC TLC Open Data)                                                   │
│        └── https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page                         │
│            • Yellow Taxi:                                                                       │
│            • Green Taxi:                                                                        │
│            • FHV:                                                                               │
│            • HVFHV:                                                                             │
│            📦 Format: Apache Parquet (Columnar, Compressed)                                     │
└─────────────────────────────────────────────────────────────────────────────────────────────────┘
                                              │
                                              │ 1️⃣ INGEST (Python + urllib + boto3)
                                              │    • Download Parquet files from source
                                              │    • Stream upload to S3 bucket
                                              │    • Organize with Hive-style partitioning
                                              ▼
┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  [STAGE 2: STORAGE - DATA LAKE]                                                                 │
│  ──────────────────────────────                                                                 │
│        🗂️ AWS S3 (Simple Storage Service)                                                       │
│        └── s3://cs341-taxi-{student_id}-bucket/                                                 │
│            └── raw_data/                                                                        │
│                ├── type=yellow/   → yellow_tripdata_2024-01.parquet                             │
│                ├── type=green/    → green_tripdata_2024-01.parquet                              │
│                ├── type=fhv/      → fhv_tripdata_2024-01.parquet                                │
│                └── type=hvfhv/    → fhvhv_tripdata_2024-01.parquet                              │
│                                                                                                 │
│        💡 Key Benefits:                                                                         │
│            • Decoupled storage (compute-independent)                                            │
│            • 99.999999999% durability (11 nines)                                                │
│            • Pay only for what you store (~$0.023/GB/month)                                     │
└─────────────────────────────────────────────────────────────────────────────────────────────────┘
                                              │
                                              │ 2️⃣ DEFINE SCHEMA (CREATE EXTERNAL TABLE)
                                              │    • DDL statements via Athena
                                              │    • No data movement (Schema-on-Read)
                                              │    • Maps S3 locations to SQL tables
                                              ▼
┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  [STAGE 3: METADATA LAYER - DATA CATALOG]                                                       │
│  ────────────────────────────────────────                                                       │
│        📚 AWS Glue Data Catalog                                                                 │
│        └── Database: taxi_analysis_db                                                           │
│            ├── Table: yellow_taxi  (19 columns: VendorID, pickup_datetime, ...)                 │
│            ├── Table: green_taxi   (20 columns: VendorID, lpep_pickup_datetime, ...)            │
│            ├── Table: fhv_taxi     (7 columns: dispatching_base_num, pickup_datetime, ...)      │
│            └── Table: hvfhv_taxi   (24 columns: hvfhs_license_num, pickup_datetime, ...)        │
│                                                                                                 │
│        💡 Key Concept: "Schema-on-Read"                                                         │
│            • Data stays in original format (Parquet)                                            │
│            • Schema applied when queried, not when stored                                       │
│            • Flexible: can redefine schema without reprocessing data                            │
└─────────────────────────────────────────────────────────────────────────────────────────────────┘
                                              │
                                              │ 3️⃣ METADATA LINK
                                              │    • Glue Catalog → Athena integration
                                              │    • Automatic schema discovery
                                              ▼
┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  [STAGE 4: COMPUTE LAYER - SERVERLESS SQL]                                                      │
│  ─────────────────────────────────────────                                                      │
│        ⚡ AWS Athena (Presto/Trino Engine)                                                       │
│        └── SQL Query Execution:                                                                 │
│            ┌──────────────────────────────────────────────────────────────────────────────┐     │
│            │  SELECT 'Yellow Taxi' as taxi_type, count(*) as rides                        │     │
│            │  FROM taxi_analysis_db.yellow_taxi                                           │     │
│            │  UNION ALL                                                                   │     │
│            │  SELECT 'Green Taxi' as taxi_type, count(*) as rides                         │     │
│            │  FROM taxi_analysis_db.green_taxi                                            │     │
│            │  ...                                                                         │     │
│            │  ORDER BY rides DESC;                                                        │     │
│            └──────────────────────────────────────────────────────────────────────────────┘     │
│                                                                                                 │
│        💡 Key Benefits:                                                                         │
│            • No servers to manage (Serverless)                                                  │
│            • Processes ~23M rows in seconds                                                     │
│            • Pay per query (~$5/TB scanned)                                                     │
│            • Parquet = less data scanned = lower cost                                           │
└─────────────────────────────────────────────────────────────────────────────────────────────────┘
                                              │
                                              │ 4️⃣ RETRIEVE RESULTS (boto3 SDK)
                                              │    • Only aggregated data returned
                                              │    • ~4 rows instead of 23M rows
                                              │    • Minimal network transfer
                                              ▼
┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  [STAGE 5: REPORTING & VISUALIZATION]                                                           │
│  ────────────────────────────────────                                                           │
│        💻 Jupyter Notebook (Local)                                                              │
│        └── Final Output:                                                                        │
│            ┌────────────────────────────────────────────────────────────┐                       │
│            │  Rank   Taxi Type      Rides Count     Percentage          │                       │
│            │  ────   ─────────      ───────────     ──────────          │                       │
│            │  1      HVFHV          20,XXX,XXX      XX.X%               │                       │
│            │  2      Yellow Taxi     2,XXX,XXX      XX.X%               │                       │
│            │  3      FHV               XXX,XXX       X.X%               │                       │
│            │  4      Green Taxi         XX,XXX       X.X%               │                       │
│            └────────────────────────────────────────────────────────────┘                       │
│                                                                                                 │
│        🏆 Answer: Which taxi type has the most rides in January 2024?                           │
└─────────────────────────────────────────────────────────────────────────────────────────────────┘

══════════════════════════════════════════════════════════════════════════════════════════════════
  🔄 DATA FLOW SUMMARY:  Source → S3 (Store) → Glue (Catalog) → Athena (Query) → Notebook (Report)
══════════════════════════════════════════════════════════════════════════════════════════════════
""")


══════════════════════════════════════════════════════════════════════════════════════════════════
                           🗺️  DATA PIPELINE MAP (NYC Taxi Data - AWS Edition)
══════════════════════════════════════════════════════════════════════════════════════════════════

┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  [STAGE 1: SOURCE]                                                                              │
│  ─────────────────                                                                              │
│        🌐 INTERNET SOURCE (NYC TLC Open Data)                                                   │
│        └── https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page                         │
│            • Yellow Taxi:                                                                       │
│            • Green Taxi:                                                                        │
│            • FHV:   

---

# 🚀 Stage 1: AWS CONFIGURATION & DEPENDENCIES

In [ ]:
# Data Pipeline Stage: SETUP
!pip install boto3 pandas botocore
import boto3
import pandas as pd
import os
import time
from botocore.exceptions import NoCredentialsError

print("⚙️ Setting up AWS Environment...")

# ==========================================
# ⚠️ UPDATE YOUR CREDENTIALS (FROM LEARNER LAB)
# ==========================================
AWS_ACCESS_KEY = "Your access key" # ใส่ Access Key
AWS_SECRET_KEY = "Your secret key" # ใส่ Secret Key
AWS_SESSION_TOKEN = "Your token session" # ใส่ Session Token
REGION_NAME = "us-east-1" # region ที่ lab กำหนด

# ⚠️ UPDATE YOUR BUCKET NAME: Must be globally unique
# Reccommend: cs341-taxi-{student_id}-bucket
BUCKET_NAME = "cs341-taxi-6609612178-bucket"

# Initialize AWS Clients
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=REGION_NAME
)

s3_client = session.client('s3')
athena_client = session.client('athena')
glue_client = session.client('glue')

print(f"✅ AWS Session Authenticated: {REGION_NAME}")
print(f"🎯 Target S3 Bucket: {BUCKET_NAME}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
⚙️ Setting up AWS Environment...
✅ AWS Session Authenticated: us-east-1
🎯 Target S3 Bucket: cs341-taxi-6609612178-bucket


# 📥 Stage 2: DATA INGESTION TO DATA LAKE (S3)
- Data Pipeline Stage: INGEST

In [2]:
import urllib.request

# URLs ข้อมูล
files_to_download = [
    ("yellow_tripdata_2024-01.parquet", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet", "yellow"),
    ("green_tripdata_2024-01.parquet", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet", "green"),
    ("fhv_tripdata_2024-01.parquet", "https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2024-01.parquet", "fhv"),
    ("fhvhv_tripdata_2024-01.parquet", "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet", "hvfhv")
]

def upload_to_s3(file_name, url, folder_prefix):
    """Download from URL and stream upload to S3 directly"""
    print(f"⬇️ Processing {file_name}...")

    # Download locally temporarily (optimization: could stream directly but keeping it simple)
    if not os.path.exists(file_name):
        urllib.request.urlretrieve(url, file_name)

    # Upload to S3 (Partitioned by type)
    s3_key = f"raw_data/type={folder_prefix}/{file_name}"
    try:
        s3_client.upload_file(file_name, BUCKET_NAME, s3_key)
        print(f"   ✅ Uploaded to s3://{BUCKET_NAME}/{s3_key}")
        # Clean up local file to save space
        os.remove(file_name)
    except Exception as e:
        print(f"   ❌ Error: {e}")

# Create Bucket if not exists (Try/Catch for existing buckets)
try:
    s3_client.create_bucket(Bucket=BUCKET_NAME)
except:
    print("ℹ️ Bucket might already exist or permission restricted (Continuing...)")

# Execute Ingestion
print("🚀 Starting Data Ingestion to AWS S3...")
for fname, url, prefix in files_to_download:
    upload_to_s3(fname, url, prefix)

print("\n🎯 Data Lake Ingestion Completed!")

🚀 Starting Data Ingestion to AWS S3...
⬇️ Processing yellow_tripdata_2024-01.parquet...
   ✅ Uploaded to s3://cs341-taxi-6609612178-bucket/raw_data/type=yellow/yellow_tripdata_2024-01.parquet
⬇️ Processing green_tripdata_2024-01.parquet...
   ✅ Uploaded to s3://cs341-taxi-6609612178-bucket/raw_data/type=green/green_tripdata_2024-01.parquet
⬇️ Processing fhv_tripdata_2024-01.parquet...
   ✅ Uploaded to s3://cs341-taxi-6609612178-bucket/raw_data/type=fhv/fhv_tripdata_2024-01.parquet
⬇️ Processing fhvhv_tripdata_2024-01.parquet...
   ✅ Uploaded to s3://cs341-taxi-6609612178-bucket/raw_data/type=hvfhv/fhvhv_tripdata_2024-01.parquet

🎯 Data Lake Ingestion Completed!


# 🔄 Stage 3: DEFINE SCHEMA (GLUE/ATHENA)
- Data Pipeline Stage: TRANSFORM / DEFINE

In [3]:
DATABASE_NAME = "taxi_analysis_db"
RESULT_OUTPUT_LOCATION = f"s3://{BUCKET_NAME}/athena_results/"

def run_athena_query(query, db=None):
    context = {'Database': db} if db else {}
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext=context,
        ResultConfiguration={'OutputLocation': RESULT_OUTPUT_LOCATION}
    )
    return response['QueryExecutionId']

def wait_for_query(query_id):
    while True:
        status = athena_client.get_query_execution(QueryExecutionId=query_id)
        state = status['QueryExecution']['Status']['State']
        if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            return state
        time.sleep(1)

print("🏗️ Setting up Data Catalog...")

# 1. Create Database
create_db_query = f"CREATE DATABASE IF NOT EXISTS {DATABASE_NAME};"
qid = run_athena_query(create_db_query, db=DATABASE_NAME) # Fixed: Pass DATABASE_NAME here
if wait_for_query(qid) == 'SUCCEEDED':
    print(f"✅ Database '{DATABASE_NAME}' ready.")

# 2. Create External Tables (Mapping S3 Parquet to SQL Table)
# Technique: Hive Partitioning or just direct mapping.
# Here we treat all files as one large dataset but categorized by the folder structure we created.
# For simplicity in this mini-challenge, we will create one table per type or a view.
# Let's create one main table and use the directory structure (Hive style) if we partitioned correctly,
# but since schemas differ slightly, we create 4 external tables.

tables_ddl = {
    "yellow_taxi": f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE_NAME}.yellow_taxi (
          VendorID bigint, tpep_pickup_datetime timestamp, tpep_dropoff_datetime timestamp, passenger_count double,
          trip_distance double, RatecodeID double, store_and_fwd_flag string, PULocationID bigint, DOLocationID bigint,
          payment_type bigint, fare_amount double, extra double, mta_tax double, tip_amount double, tolls_amount double,
          improvement_surcharge double, total_amount double, congestion_surcharge double, Airport_fee double
        ) STORED AS PARQUET LOCATION 's3://{BUCKET_NAME}/raw_data/type=yellow/'
    """,
    "green_taxi": f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE_NAME}.green_taxi (
          VendorID bigint, lpep_pickup_datetime timestamp, lpep_dropoff_datetime timestamp, store_and_fwd_flag string,
          RatecodeID double, PULocationID bigint, DOLocationID bigint, passenger_count double, trip_distance double,
          fare_amount double, extra double, mta_tax double, tip_amount double, tolls_amount double, ehail_fee double,
          improvement_surcharge double, total_amount double, payment_type double, trip_type double, congestion_surcharge double
        ) STORED AS PARQUET LOCATION 's3://{BUCKET_NAME}/raw_data/type=green/'
    """,
    "fhv_taxi": f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE_NAME}.fhv_taxi (
          dispatching_base_num string, pickup_datetime timestamp, dropOff_datetime timestamp,
          PUlocationID double, DOlocationID double, SR_Flag double, Affiliated_base_number string
        ) STORED AS PARQUET LOCATION 's3://{BUCKET_NAME}/raw_data/type=fhv/'
    """,
    "hvfhv_taxi": f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE_NAME}.hvfhv_taxi (
           hvfhs_license_num string, dispatching_base_num string, originating_base_num string, request_datetime timestamp,
           on_scene_datetime timestamp, pickup_datetime timestamp, dropoff_datetime timestamp, PULocationID bigint,
           DOLocationID bigint, trip_miles double, trip_time bigint, base_passenger_fare double, tolls double,
           bcf double, sales_tax double, congestion_surcharge double, airport_fee double, tips double, driver_pay double,
           shared_request_flag string, shared_match_flag string, access_a_ride_flag string, wav_request_flag string, wav_match_flag string
        ) STORED AS PARQUET LOCATION 's3://{BUCKET_NAME}/raw_data/type=hvfhv/'
    """
}

for name, ddl in tables_ddl.items():
    print(f"   Creating table definition for: {name}")
    qid = run_athena_query(ddl, DATABASE_NAME)
    wait_for_query(qid)

print("✅ Data Catalog Definitions Completed.")

🏗️ Setting up Data Catalog...
✅ Database 'taxi_analysis_db' ready.
   Creating table definition for: yellow_taxi
   Creating table definition for: green_taxi
   Creating table definition for: fhv_taxi
   Creating table definition for: hvfhv_taxi
✅ Data Catalog Definitions Completed.


# 📊 Stage 4: AGGREGATE ON CLOUD (ATHENA)
# Data Pipeline Stage: AGGREGATE

In [4]:
print("⚡ Running Serverless SQL Query on AWS Athena...")

# SQL Query: Count rows from each table and combine results
# This runs on AWS infrastructure, not your notebook kernel.
aggregate_query = f"""
SELECT 'Yellow Taxi' as taxi_type, count(*) as rides FROM {DATABASE_NAME}.yellow_taxi
UNION ALL
SELECT 'Green Taxi' as taxi_type, count(*) as rides FROM {DATABASE_NAME}.green_taxi
UNION ALL
SELECT 'FHV' as taxi_type, count(*) as rides FROM {DATABASE_NAME}.fhv_taxi
UNION ALL
SELECT 'HVFHV' as taxi_type, count(*) as rides FROM {DATABASE_NAME}.hvfhv_taxi
ORDER BY rides DESC;
"""

# Execute Query
query_id = run_athena_query(aggregate_query, DATABASE_NAME)
print(f"⏳ Query Execution ID: {query_id}")

# Wait for completion
status = wait_for_query(query_id)
if status == 'SUCCEEDED':
    print("✅ Query Succeeded!")

    # Get Results
    results_paginator = athena_client.get_query_results(QueryExecutionId=query_id)

    # Convert Athena Result to Pandas DataFrame
    # (Handling the complex Athena JSON response)
    rows = []
    for page in results_paginator['ResultSet']['Rows']:
        rows.append([col.get('VarCharValue', '') for col in page['Data']])

    # First row is header
    df_results = pd.DataFrame(rows[1:], columns=rows[0])
    # Convert types
    df_results['rides'] = df_results['rides'].astype(int)

else:
    print(f"❌ Query Failed: {status}")

⚡ Running Serverless SQL Query on AWS Athena...
⏳ Query Execution ID: b3aadcac-bdfb-482c-a02a-e361fa5ac3c7
✅ Query Succeeded!


# 📋 Stage 5: REPORT & VISUALIZATION
# Data Pipeline Stage: REPORT

In [5]:
print("📈 FINAL RESULTS - NYC Taxi Rides Analysis (Cloud Edition)")
print("=" * 70)

# Calculate Percentage
total_rides = df_results['rides'].sum()
df_results['percentage'] = (df_results['rides'] / total_rides) * 100

# Display Table
print(f"{'Rank':<6} {'Taxi Type':<15} {'Rides Count':<15} {'Percentage':<10}")
print("-" * 55)

for idx, row in df_results.iterrows():
    print(f"{idx+1:<6} {row['taxi_type']:<15} {row['rides']:>12,} {row['percentage']:>8.1f}%")

print("-" * 55)
print(f"{'TOTAL':<22} {total_rides:>12,} {'100.0%':>8}")

top_type = df_results.iloc[0]
print(f"\n🏆 Top taxi type (Jan 2024): {top_type['taxi_type']} — {top_type['rides']:,} rides")

print("\n💡 AWS Architecture Benefits:")
print("1. Scalability: S3 can hold PB of data.")
print("2. Serverless: No EC2 instances were managed.")
print("3. Cost: Paid only for S3 storage & query scan time.")

📈 FINAL RESULTS - NYC Taxi Rides Analysis (Cloud Edition)
Rank   Taxi Type       Rides Count     Percentage
-------------------------------------------------------
1      HVFHV             19,663,930     82.0%
2      Yellow Taxi        2,964,624     12.4%
3      FHV                1,290,116      5.4%
4      Green Taxi            56,551      0.2%
-------------------------------------------------------
TOTAL                    23,975,221   100.0%

🏆 Top taxi type (Jan 2024): HVFHV — 19,663,930 rides

💡 AWS Architecture Benefits:
1. Scalability: S3 can hold PB of data.
2. Serverless: No EC2 instances were managed.
3. Cost: Paid only for S3 storage & query scan time.


# 🎯 **สรุปคำตอบสุดท้าย (FINAL CONCLUSION)**

## **คำตอบโจทย์ที่ชัดเจน:**
# A. Pipeline Architecture Diagram: Desbribe how to work
### ⚙️ How it Works: The Serverless Data Pipeline
อธิบายกระบวนการทำงานตาม Data Flow 5 ขั้นตอน (Ingest → Store → Clean/Transform → Query/Aggregate → Output):

**1. Ingest (นำเข้าข้อมูล):**
* ใช้ Python Script (`boto3`) ทำหน้าที่เป็น Ingestion Layer
* ดาวน์โหลดข้อมูลดิบ (Parquet) จาก NYC Taxi Source และทำการ **Stream Upload** ขึ้นสู่ AWS S3 ทันทีโดยไม่ต้องพักข้อมูลลง Disk เครื่อง local นาน

**2. Store (จัดเก็บข้อมูล):**
* ข้อมูลทั้งหมดถูกจัดเก็บใน **AWS S3** (Data Lake) ในรูปแบบ **Partitioned Folder** (`/type=yellow`, `/type=green`)
* การเก็บแบบนี้แยกส่วน Storage ออกจาก Compute (Decoupled Storage) ทำให้ต้นทุนต่ำและขยายขนาดได้ไม่จำกัด

**3. Clean/Transform (ทำความสะอาดและแปลงรูป):**
* ใช้หลักการ **Schema-on-Read** ผ่าน **AWS Glue Data Catalog**
* แทนที่จะเสียเวลาแปลงไฟล์ (ETL) เราใช้การสร้าง **External Table** เพื่อ "กำหนดโครงสร้าง" (Define Schema) ครอบลงไปบนไฟล์ดิบ ทำให้เสมือนว่าข้อมูลถูก Clean และ Transform เป็นตารางพร้อมใช้แล้ว

**4. Query/Aggregate (สอบถามและรวบรวมผล):**
* ใช้ **AWS Athena** (Serverless SQL Engine) เป็นตัวประมวลผล
* ส่งคำสั่ง SQL (`SELECT count(*) ...`) ไปทำงานบน Cloud ซึ่ง Athena จะทำการสแกนไฟล์บน S3 และรวบรวมผลลัพธ์ (Aggregation) อย่างรวดเร็ว

**5. Output (ผลลัพธ์):**
* ผลลัพธ์สุดท้าย (Aggregated Results) ที่มีขนาดเล็กจะถูกส่งกลับมายัง **Jupyter Notebook**
* ใช้ **Pandas** ในการจัดเรียงลำดับ (Ranking) และแสดงผลเป็นตารางสรุปคำตอบ (Final Answer)
---
# B. Final Result
### 📊 **ตารางสรุปจำนวน rides (เรียงจากมากไปน้อย)**
| อันดับ | ประเภท Taxi | จำนวน Rides | เปอร์เซ็นต์ |
|--------|-------------|-------------|-----------|
| 1      | **HVFHV**   | **19,663,930** | **82.0%** |
| 2      | Yellow Taxi | 2,964,624   | 12.4%     |
| 3      | FHV         | 1,290,116   | 5.4%      |
| 4      | Green Taxi  | 56,551      | 0.2%      |

### 📝 **ข้อความสรุป**
**🏆 Top taxi type (Jan 2024): HVFHV — 19,663,930 rides**

---
# C. Reflection
### 1. Comparison: Before (Local) vs. After (Cloud)
| Aspect | Before (Local / DuckDB) | After (AWS Cloud / S3 + Athena) |
| :--- | :--- | :--- |
| **Storage** | เก็บใน Hard Disk เครื่องตัวเอง (กินพื้นที่) | เก็บใน **S3 Data Lake** (รองรับระดับ Petabyte) |
| **Compute** | ใช้ RAM/CPU เครื่องตัวเอง (ถ้าข้อมูลใหญ่ เครื่องค้าง) | ใช้ **Athena Serverless** (ประมวลผลบน Cloud เร็วและแรง) |
| **Scalability** | ทำงานได้จำกัดแค่สเปคเครื่องที่มี | **Unlimited Scalability** รองรับข้อมูลมหาศาลได้ทันที |
| **Cost** | ฟรี (แต่เสียค่าไฟ/ค่าเครื่อง) | **Pay-as-you-go** จ่ายตามพื้นที่เก็บและปริมาณข้อมูลที่ Scan |

### 2. What was the most difficult part?
* **AWS Configuration & IAM:** ความยากที่สุดคือการจัดการเรื่อง **Credentials (Access Key/Secret Key)** และการทำความเข้าใจเรื่อง Permission ว่าต้องอนุญาตให้ Athena เข้าถึง S3 ได้ อย่างไรก็ตาม การใช้ **Learner Lab** ช่วยลดความซับซ้อนเรื่อง Permission ลงไปได้บ้าง แต่ต้องคอยระวังเรื่อง Session Token หมดอายุ
* **Schema-on-Read Concept:** การปรับความเข้าใจว่าเราไม่ต้อง "Create Table" แบบ Database ทั่วไปที่มีข้อมูลอยู่ข้างใน แต่เป็นการชี้ (Point) ไปที่ S3 แทน ทำให้ต้องระวังเรื่อง Path ของข้อมูลให้ถูกต้องแม่นยำ

### 3. What did I learn?
* **Modern Data Architecture:** ได้เรียนรู้โครงสร้าง **Data Lakehouse** ที่แยก Storage (S3) ออกจาก Compute (Athena) อย่างชัดเจน
* **ELT Process:** เข้าใจกระบวนการ **Extract-Load-Transform** ที่เน้นเอาข้อมูลขึ้น Cloud ให้เร็วก่อน (Load to S3) แล้วค่อยใช้พลังของ Cloud มาจัดการแปลงข้อมูลทีหลัง (Query via Athena)
* **Serverless Power:** เห็นภาพชัดเจนว่า Serverless ช่วยลดงาน Maintenance (No-Ops) และประหยัดค่าใช้จ่ายสำหรับงาน Big Data ได้จริง
